
「TorchScript入門」
===============================================================
【原題】INTRODUCTION TO TORCHSCRIPT

【原著者】James Reed (jamesreed@fb.com), Michael Suo (suo@fb.com)

【元URL】https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2020年10月24日

【チュトーリアル概要】

本チュートリアルでは、PyTorchモデルのモデル構造について簡単に説明したあと、TorchScriptと呼ばれる手法について、概要と使用方法を解説します。

---


本チュートリアルでは、TorchScriptについて紹介します。

TorchScriptは``nn.Module``から構成されたPyTochモデルの中間表現であり、C++のような高速な実行環境でモデルを実行することができます。



本チュートリアルでは、以下の内容を解説します。

1. PyTorchで作られモデルの基礎的内容、とくに
- Module（モジュール）
-  ``forward`` 関数
- モジュールを階層的に変換する方法

<br>

2. 特定の手法でPyTorchのモジュールを、高性能なデプロイ環境で実行可能なTorchScripに変換する方法、とくに

- モジュールをトレースする方法（Tracing)
- コードを直接モジュールにするスクリプトの方法（scripting）
- 上記2つの方法の相互利用と同時利用の方法
- TorchScriptモジュールの保存とロード方法




本チュートリアルが終了した際には、ぜひこの次のチュートリアルである、「C++でのTorchScriptモデルのロード手法」（LOADING A TORCHSCRIPT MODEL IN C++）にも挑戦してみてください。

次のチュートリアルでは、C++からTorchScriptを実行する方法について解説しています。

In [1]:
%matplotlib inline
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.7.0+cu101


PyTochモデルの基本構成
---------------------------------

簡単なモジュールを定義してみましょう。

``Module``はPyTorchを構成する基本ユニットです。

``Module``には以下の内容が含まれています。

1. モジュールを呼び出すための準備となるコンストラクタ
2. 一連の``Parameters`'と、サブ``Modules``。なおこれらのパラメータとサブモジュールは、モジュールが呼び出されてコンストラクタで初期化された後に、使用できるようになります。
3. 順伝搬である``forward``関数。モジュールそのものが呼び出されたときに実行される関数です。

まずは簡単な例を確認してみましょう。



In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.4920, 0.8299, 0.5254, 0.8509],
        [0.8504, 0.8406, 0.9022, 0.6847],
        [0.6422, 0.8253, 0.7027, 0.5935]]), tensor([[0.4920, 0.8299, 0.5254, 0.8509],
        [0.8504, 0.8406, 0.9022, 0.6847],
        [0.6422, 0.8253, 0.7027, 0.5935]]))


上記では

1. ``torch.nn.Module``のサブクラスを作成しました。
2. コンストラクタを定義しました。
コンストラクタは特に多くの処理をしているわけではなく、``super``で親クラスのコンストラクタを実行しているだけです。
3. 順伝搬関数``forward``を定義しました。これは2つのパラメータを入力として受け取り、2つの変数を出力します。本チュートリアルでは順伝搬関数の中身は重要ではありませんが、一種の[RNN](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)的な動作を想定し、RNNの一部を実装しました。

そして、このモジュールをインスタンス化し、3x4の乱数行列である変数``x``と``y``を用意しました。

そして、モジュールを``my_cell(x, h)``で呼び出して実行しています。このモジュールを呼び出して実行すると、関数``forward``が呼び出されて実行されることになります。

続いて、さらに面白いことに挑戦してみましょう。



In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.7103,  0.1837,  0.2306,  0.5142],
        [ 0.8638,  0.4461,  0.6245,  0.6464],
        [ 0.6585, -0.0320,  0.7657,  0.0201]], grad_fn=<TanhBackward>), tensor([[ 0.7103,  0.1837,  0.2306,  0.5142],
        [ 0.8638,  0.4461,  0.6245,  0.6464],
        [ 0.6585, -0.0320,  0.7657,  0.0201]], grad_fn=<TanhBackward>))


再度、モジュール``MyCell``を定義しなおしました。

今度は、``self.linear``のメンバ変数を追加し、``forward``関数内でこの``self.linear``を呼び出しています。

このとき、実際には何が起こっているのでしょうか？




``torch.nn.Linear``はPytorchの標準ライブラリで用意されている``Module``です、この ``MyCell``と同じようなものになります。

この``torch.nn.Linear``はモジュール内の関数で、forwardなどをそのまま呼び出すことができます。

結果、モジュールの階層的な構造を構成することができました。

print文でモジュールを出力すると、このモジュールのサブクラスの仮想的表現を確認することができます。

（print(my_cell)の部分）

この例では、サブクラス``Linear``とそのパラメータを確認できています。

このように``Module``を複数使用して構成することで、再利用可能なコンポーネントをから成るモデルを簡潔かつ読みやすく実装することができます。

ここで上記セルの、出力結果にある``grad_fn``の記載が気になるかと思います。

``grad_fn``はPyTorchの自動微分、[`autograd<>`](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)、から詳細をご覧ください。

簡単にはこの自動微分により、複雑なプログラムの偏微分を計算することができます。

PyTorchではこの自動微分がうまく設計され、組み込まれているために、モデル実装の柔軟性が大幅に向上しています。

続ていて、その柔軟性を確かめてみましょう。




In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.2030,  0.1072, -0.0427,  0.7238],
        [ 0.2365,  0.5272,  0.3636,  0.8485],
        [-0.0170,  0.3070,  0.7457,  0.4996]], grad_fn=<TanhBackward>), tensor([[ 0.2030,  0.1072, -0.0427,  0.7238],
        [ 0.2365,  0.5272,  0.3636,  0.8485],
        [-0.0170,  0.3070,  0.7457,  0.4996]], grad_fn=<TanhBackward>))


再度、``MyCell``クラスを定義しなおしました。

今回は、一緒に``MyDecisionGate``クラスも定義しています。このモジュールは順伝搬時の流れを制御するために使用します（**control flow**）。

多くの場合、順伝搬を制御するために、ループ構文やif文を含んでいます。

多くのディープラーニング・フレームワークは完全なモデル表現を必要とします（日本語訳注：そのため**control flow**が使えません）。

<br>

ですが、PyTorchの場合、勾配計算を連続した記録として扱えます。

PyTorchでは実際に計算時に何が行われたのかを記録し、出力の偏微分を求める際にはその記録を逆戻しして計算します。

そのためPyTorchでは、事前に完全にモデルの逆伝搬を定義しておく必要がありません。

<br>

（日本語訳注：このようなPyTorchの特徴はDefine by Runと呼ばれます。一方で先に計算のフローを固定していまう手法は、Define and Runと呼ばれます）

<br>


自動微分の働きについては以下のアニメーション図も参考にしてみてください。

図. [自動微分の動作の様子](https://github.com/pytorch/pytorch/raw/master/docs/source/_static/img/dynamic_graph.gif)


TorchScriptの基本
---------------------

それでは、サンプルの実行方法と、TorchScriptの使用方法を学んでいきましょう。

一言で表すと、TorchScriptはPyTorchの柔軟で動的な性質を考慮しながら、モデルをキャプチャするツールです。

まずは、トレース（tracing）による、モジュールのキャプチャを試しましょう。

In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[-0.2750,  0.6314,  0.5681,  0.6225],
         [ 0.0644, -0.1122,  0.5933,  0.3037],
         [-0.2965,  0.5491,  0.1874,  0.2215]], grad_fn=<TanhBackward>),
 tensor([[-0.2750,  0.6314,  0.5681,  0.6225],
         [ 0.0644, -0.1122,  0.5933,  0.3037],
         [-0.2965,  0.5491,  0.1874,  0.2215]], grad_fn=<TanhBackward>))

本チュートリアルの2番目に作成した``MyCell``クラスを再定義しています。

このクラスの順伝搬を実行する前に、``torch.jit.trace``を使用し、モジュールへの入力の例を渡しています。

<br>

（日本語訳注：分かりづらいのですが、

traced_cell = torch.jit.trace(my_cell, (x, h))

の部分です。my_cellで順伝搬させず、入力のサンプルとしてx,hと、そしてモデルmy_cellをtorch.jit.traceに渡して、出力traced_cellを得ています。）




ここで実際には何が行われたのでしょうか。

``torch.jit.trace``は``Module``クラスを引数に持っており、このモジュールの順伝搬を実行し、``TracedModule``のインスタンスである``torch.jit.ScriptModule``であるtraced_cellを作成しました。

TorchScriptは中間表現（IR：Intermediate Representation）を記録します。

この中間表現はディープラーニングの界隈ではよく、グラフ（graph）とも呼ばれます。

このグラフは、 ``.graph``プロパティを呼び出すことで確認することができます。

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %input : Float(3:4, 4:1, requires_grad=0, device=cpu),
      %h : Float(3:4, 4:1, requires_grad=0, device=cpu)):
  %19 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %21 : Tensor = prim::CallMethod[name="forward"](%19, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-5-1f6e08af67d0>:7:0
  %13 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::add(%21, %h, %12) # <ipython-input-5-1f6e08af67d0>:7:0
  %14 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::tanh(%13) # <ipython-input-5-1f6e08af67d0>:7:0
  %15 : (Float(3:4, 4:1, requires_grad=1, device=cpu), Float(3:4, 4:1, requires_grad=1, device=cpu)) = prim::TupleConstruct(%14, %14)
  return (%15)



しかしながら、中間表現は非常に低レベルな表現手法（人間には分かりづらく、計算器に扱いやすいレベルの表現手法）です。

そのため、グラフ情報の大半はユーザーにとって有用なものではありません。

その代わり、``.code``プロパティを使用することで、TorchScriptのコード（グラフ）を解釈しやすいように出力することが可能です。


In [7]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)
  _1 = torch.tanh(_0)
  return (_1, _1)



ここまでサンプルを実行しましたが、なんの目的があって、このような操作を行っているのでしょうか？

それにはいくつかの理由（利点）はいくつか存在します。

1. TorchScriptのコードはPythonのインタプリタではない、独自のインタプリタで呼び出されます。この独自のインタプリタは、Pythonインタプリタとは異なり、Global Interpreter Lockにひっかからないので、同じインスタンスを、同時に複数のリクエストで処理することができます。

2. TorchScriptのフォーマットはモデルの全情報を保存し、異なる環境、例えばPython以外の言語で書かれたサーバーでもロードが可能です。

3. TorchScriptはコンパイラに最適化されており、実行が、より効率的になります。

4. TorchScriptは様々なバックエンドやデバイス環境で実行可能であり、個別の実行演算を持つ各種プログラムにモデルを変換するよりも適用範囲が広いです。




``traced_cell``を呼び出すことでも、元のモジュールのPythonインスタンス``my_cell``を呼び出した結果と同じ内容を得ることができます。

In [ ]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.6958,  0.1292,  0.0430,  0.1840],
        [ 0.7851,  0.2323, -0.2598,  0.2947],
        [ 0.5422, -0.1874, -0.4077, -0.0105]], grad_fn=<TanhBackward>), tensor([[ 0.6958,  0.1292,  0.0430,  0.1840],
        [ 0.7851,  0.2323, -0.2598,  0.2947],
        [ 0.5422, -0.1874, -0.4077, -0.0105]], grad_fn=<TanhBackward>))
(tensor([[ 0.6958,  0.1292,  0.0430,  0.1840],
        [ 0.7851,  0.2323, -0.2598,  0.2947],
        [ 0.5422, -0.1874, -0.4077, -0.0105]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.6958,  0.1292,  0.0430,  0.1840],
        [ 0.7851,  0.2323, -0.2598,  0.2947],
        [ 0.5422, -0.1874, -0.4077, -0.0105]],
       grad_fn=<DifferentiableGraphBackward>))


TorchScriptを利用したモジュールの変換
----------------------------------

先ほどの実装コードで、の2番目の実装例`MyCell`を使用し、3番目の
control-flow-ladenサブモジュール（control flowモジュールを含むもの）を使わなかったのには理由があります。

その理由を確認してみましょう。



In [8]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.dg
  _1 = (self.linear).forward(input, )
  _2 = (_0).forward(_1, )
  _3 = torch.tanh(torch.add(_1, h, alpha=1))
  return (_3, _3)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


``.code``の出力をご覧ください。if-else文が見当たりません。

なぜでしょうか。

<br>

トレースは実際にコードが実行された通りに、その操作を記録し、TorchScriptのScriptModuleを生成します。

そのため残念ながら、制御部分（control flow）は考慮できません。

それではTorchScriptにcotrol flowを盛り込むにはどうすれば良いのでしょうか。



ここで、PythonのソースコードをTorchScriptへと解析、変換する**script compiler**（スクリプト・コンパイラ）を使用します。

``MyDecisionGate``クラスを変換してみましょう。



In [9]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



（日本語訳注）

上記では（self.dg）という部分が出力に含まれており、それがcontrol flowになっています。

ですが、分かりづらいので、一応、TorchScrpitになったscripted_gateを確認しておきましょう。

In [10]:
# 日本語版追記 TorchScriptにif文が入っていることが分かります
print(scripted_gate.code)

def forward(self,
    x: Tensor) -> Tensor:
  _0 = bool(torch.gt(torch.sum(x, dtype=None), 0))
  if _0:
    _1 = x
  else:
    _1 = torch.neg(x)
  return _1



これで、うまくいきました。

以上により、TorchScriptにcontrol flowを組み込めたので、実際に実行してみます。




In [11]:
# New inputs
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[ 0.5665,  0.3799,  0.7255, -0.0325],
         [ 0.8528,  0.5922,  0.7562, -0.0273],
         [ 0.1272,  0.2416,  0.7779,  0.7921]], grad_fn=<TanhBackward>),
 tensor([[ 0.5665,  0.3799,  0.7255, -0.0325],
         [ 0.8528,  0.5922,  0.7562, -0.0273],
         [ 0.1272,  0.2416,  0.7779,  0.7921]], grad_fn=<TanhBackward>))

**ScrptingとTracingを同時に使用する方法**

ときに、PyTorchのモジュールが複雑であり、control flowが多様で、トレース（tracing）ではなくスクリプト（scripting）で変換したい場合があります。

このような場合には、``torch.jit.script`をインラインで使用することで、モジュールのトレーシングと同時に使用することができます。

最初の例を見てみましょう。

（``torch.jit.trace``がインラインで使われた後、scriptしています）



In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



次の例を確認してみましょう。

（上記コードのクラスMyRNNLoopを``torch.jit.script``のインラインで使用）



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, h, = (self.loop).forward(argument_1, )
  return torch.relu(h)



このように、スクリプト（scripting）とトレース（tracing）は、どちらかからどちらかのTorchScriptを呼び出したり、同時に使用したりできます。

TorchScriptでのモデルの保存とロード
-------------------------

最後に、TorchScriptのモジュールを、ディスクにアーカイブフォーマット（archive format）で保存、ロードする関数（API）を解説します。

このフォーマットには、コード、パラメータ、属性、およびデバッグ情報が含まれており、アーカイブフォーマットは、完全に別のプロセスでロードできる独立した表現手法です。

さきほどのRNNモデルを保存、ロードしてみましょう。




In [14]:
traced.save('wrapped_rnn.zip')

loaded = torch.jit.load('wrapped_rnn.zip')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, h, = (self.loop).forward(argument_1, )
  return torch.relu(h)



ご覧の通り、変換は元のモジュールの構造を保っており、実行可能です。

このモデルはPythonの実行環境に限らず例えば、C++実行環境でもロードすることができます。


**さらなる発展へ**

以上で本チュートリアルは終了です。

<br>

さらなるサンプル例としては、

「NeurIPS demo for converting machine translation models using
TorchScript」

のGoogle Colabコードもぜひご覧ください。

https://colab.research.google.com/drive/1HiICg6jRkBnr5hvK2-VnMi88Vi9pUzEJ


